In [37]:
#Webscrape https://www.rebellegirls.com/yarnfiber.htm for list of offered brands
import requests
from bs4 import BeautifulSoup
import pandas as pd
from APIkey import APIkey, APIpw
import json
import csv

In [11]:
url= "https://www.rebellegirls.com/yarnfiber.htm"
page= requests.get(url)

soup=BeautifulSoup(page.content, "html.parser")
results=soup.find(id="accordion")
   
#used ChatGPT to figure out this block of webscraping code
stock=[]
name=[]
brands = soup.find_all('h3')
for brand in brands:
    brand_name = brand.text.strip()
    yarn_list = brand.find_next('div').find_all('li')
    for yarn in yarn_list:
        yarn_name = yarn.text.strip()
        name.append(yarn_name)
        stock.append(brand_name)


In [12]:
#create database
stock_list={"stock":stock, "name":name}
rebelle=pd.DataFrame(stock_list)

rebelle['combined']=rebelle['stock']+ '+'+ rebelle['name']
print(rebelle['combined'])


0          Adriafil+Knitcol
1             Anzula+Breeze
2              Anzula+Cloud
3              Anzula+Haiku
4            Anzula+Lunaris
               ...         
125        Add-Ins+Firestar
126        Add-Ins+Angelina
127        Add-Ins+Soffsilk
128    Add-Ins+Silk Cocoons
129      Add-Ins+Silk Noils
Name: combined, Length: 130, dtype: object


In [45]:
url = 'https://api.ravelry.com/patterns/search.json?yarn=Adriafil+Knitcol'

yarn= 'Adriafil+Knitcol'


    

response=requests.get(url, auth= (APIkey, APIpw))

response

<Response [200]>

In [47]:
rav_things=response.json()
patterns= rav_things.get("patterns")

In [48]:
def write_json(data, filename= "ravelry_api.json"):
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

data=(patterns)
write_json(data)

In [24]:
def append_json():
    with open('ravelry_api.json', 'r') as file:
        
        data= json.load(file)
          
        data.update(new_data)
        
    with open('ravelry_api.json', 'a+') as file:
        json.dump(data, file)
        

In [49]:
with open('ravelry_api.json') as f:
        
    data= json.load(f)
    
df= pd.DataFrame(data)

print(df)

     free       id                         name                  permalink  \
0    True  7378256                     Mug Cozy                mug-cozy-31   
1   False   706936              Puff Stitch Hat          puff-stitch-hat-9   
2    True   650697         Toe-Up Worsted Socks       toe-up-worsted-socks   
3    True    93685    Child's Self-striping Hat   childs-self-striping-hat   
4    True   545271             Best Baby Beanie           best-baby-beanie   
..    ...      ...                          ...                        ...   
95   True   596470     Simple Coat Hanger Cover   simple-coat-hanger-cover   
96  False   599187  Football Hat (sport weight)  football-hat-sport-weight   
97  False   599270   Tricot's First Dog Sweater  tricots-first-dog-sweater   
98   True   605701           Adriafil Monet set         adriafil-monet-set   
99  False   605713         Adriafil Bach Poncho       adriafil-bach-poncho   

   personal_attributes                                        f

In [ ]:
with open('ravelry_api.json') as f:
    data= json.loads(f)

patterns= data[0]

anzula= open('anzula.csv', 'w')
csv_writer= csv.writer(anzula)

count = 0

for pat in patterns:
    if count == 0:
        header=patterns.keys()
        csv_writer.writerow(header)
        count+=1
    
    csv_writer.writerow(patterns.values())

anzula.close()